In [68]:
%load_ext nb_black
%load_ext lab_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black
The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


<IPython.core.display.Javascript object>

In [69]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

<IPython.core.display.Javascript object>

In [70]:
books_filename = "BX-Books.csv"
ratings_filename = "BX-Book-Ratings.csv"

<IPython.core.display.Javascript object>

In [71]:
# import csv data into dataframes
books = pd.read_csv("BX-Books.csv", sep=";", error_bad_lines=False, encoding="latin-1")
books.columns = [
    "ISBN",
    "bookTitle",
    "bookAuthor",
    "yearOfPublication",
    "publisher",
    "imageUrlS",
    "imageUrlM",
    "imageUrlL",
]
users = pd.read_csv("BX-Users.csv", sep=";", error_bad_lines=False, encoding="latin-1")
users.columns = ["userID", "Location", "Age"]
ratings = pd.read_csv(
    "BX-Book-Ratings.csv", sep=";", error_bad_lines=False, encoding="latin-1"
)
ratings.columns = ["userID", "ISBN", "bookRating"]

C:\Users\makro\AppData\Local\Temp\ipykernel_6880\3670924125.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books = pd.read_csv("BX-Books.csv", sep=";", error_bad_lines=False, encoding="latin-1")
b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: 

<IPython.core.display.Javascript object>

In [72]:
combine_book_rating = pd.merge(ratings, books, on="ISBN")
columns = [
    "yearOfPublication",
    "publisher",
    "bookAuthor",
    "imageUrlS",
    "imageUrlM",
    "imageUrlL",
]
combine_book_rating = combine_book_rating.drop(columns, axis=1)

<IPython.core.display.Javascript object>

In [73]:
combine_book_rating = combine_book_rating.dropna(axis=0, subset=["bookTitle"])

book_ratingCount = (
    combine_book_rating.groupby(by=["bookTitle"])["bookRating"]
    .count()
    .reset_index()
    .rename(columns={"bookRating": "totalRatingCount"})[
        ["bookTitle", "totalRatingCount"]
    ]
)

rating_with_totalRatingCount = combine_book_rating.merge(
    book_ratingCount, left_on="bookTitle", right_on="bookTitle", how="left"
)

popularity_threshold = 50
rating_popular_book = rating_with_totalRatingCount.query(
    "totalRatingCount >= @popularity_threshold"
)

<IPython.core.display.Javascript object>

In [74]:
from scipy.sparse import csr_matrix

combined = rating_popular_book.merge(
    users, left_on="userID", right_on="userID", how="left"
)
combined = combined.drop_duplicates(["userID", "bookTitle"])

combined_pivot = combined.pivot(
    index="bookTitle", columns="userID", values="bookRating"
).fillna(0)
combined_matrix = csr_matrix(combined_pivot.values)

<IPython.core.display.Javascript object>

In [75]:
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric="cosine", algorithm="brute")
model_knn.fit(combined_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

<IPython.core.display.Javascript object>

In [76]:
from operator import itemgetter


def get_recommends(book):
    recommendations = []
    x = combined_pivot.loc[[book]]
    distances, indices = model_knn.kneighbors(x.values.reshape(1, -1), n_neighbors=6)

    for i in range(0, len(distances.flatten())):
        if i == 0:
            pass
        else:
            recommendations.append(
                [combined_pivot.index[indices.flatten()[i]], distances.flatten()[i]]
            )
    return [book, sorted(recommendations, key=itemgetter(1), reverse=True)]


print(get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))"))

['The Queen of the Damned (Vampire Chronicles (Paperback))', [['Pandora: New Tales of the Vampires (New Tales of the Vampires)', 0.8440147701469674], ['Memnoch the Devil (Vampire Chronicles, No 5)', 0.7420628214446703], ['Interview with the Vampire', 0.7202435286233742], ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.595377238255751], ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5620454125950052]]]


<IPython.core.display.Javascript object>

Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [77]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)


def test_book_recommendation():
    test_pass = True
    recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
    if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
        test_pass = False
    recommended_books = [
        "Saving Grace",
        "The Pilot's Wife : A Novel",
        "I Know This Much Is True",
        "Blue Diary",
        "The Lovely Bones: A Novel",
    ]
    recommended_books_dist = [0.92, 0.91, 0.91, 0.91, 0.90]
    for i in range(2):
        if recommends[1][i][0] not in recommended_books:
            test_pass = False
        if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
            test_pass = False
    if test_pass:
        print("You passed the challenge! 🎉🎉🎉🎉🎉")
    else:
        print("You haven't passed yet. Keep trying!")


test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['Saving Grace', 0.9201843021441911], ["The Pilot's Wife : A Novel", 0.9172570741861605], ['I Know This Much Is True', 0.9101475435893542], ['Blue Diary', 0.9101138470509211], ['The Lovely Bones: A Novel', 0.9095981950904724]]]
You passed the challenge! 🎉🎉🎉🎉🎉


<IPython.core.display.Javascript object>